In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
%matplotlib inline 
# to put the graphs directly inder the cells instead of in a pop-up

UsageError: unrecognized arguments: #to put the graphs directly inder the cells instead of in a pop-up


In [ ]:
# ==========================================
# 1. DATA LOADING
# ==========================================

# Définition des noms de colonnes selon la documentation NASA (C-MAPSS)
index_names = ['unit_nr', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['s_{}'.format(i) for i in range(1, 22)] # Génère s_1 à s_21
col_names = index_names + setting_names + sensor_names

# Chargement des données
# sep='\s+' signifie "n'importe quel nombre d'espaces" (regex)
print("Loading data from C-MAPSS dataset...")
df_train = pd.read_csv('data/train_FD001.txt', sep='\s+', header=None, names=col_names)
df_test = pd.read_csv('data/test_FD001.txt', sep='\s+', header=None, names=col_names)
y_test = pd.read_csv('data/RUL_FD001.txt', sep='\s+', header=None, names=['RUL'])

print("Data loaded successfully!")
print(f"Training Set shape: {df_train.shape}")
print(f"Test Set shape:     {df_test.shape}")

# Aperçu des 5 premières lignes
df_train.head()